# VVADLRS3 Pretrained Models live in action


## Init Variables

In [1]:
from vvadlrs3 import sample, dlibmodels
import numpy as np
import dlib
from dvg_ringbuffer import RingBuffer

# create a sample
k = 36  # Number of frames used for inference
shape = (96, 96)  # Resolution of the input imgae for the prediction
featureType = 'faceImage'  # Type of the features that will be created from the Image
# TODO: this should actually only be needed if not using faceImage type
shapeModelPath = str(dlibmodels.SHAPE_PREDICTOR_68_FACE_LANDMARKS())
ffg = sample.FaceFeatureGenerator(
    featureType, shapeModelPath=shapeModelPath, shape=shape)

# TODO: Fist approach only with a detector - later we can try FaceTracker for multiple faces?
detector = dlib.get_frontal_face_detector()

# Ringbuffer for features
rb = RingBuffer(36, dtype=(np.uint8, (96, 96, 3)))

samples = []

## Load Pretrained Model

In [2]:
from vvadlrs3 import pretrained_models

model = pretrained_models.getFaceImageModel()

print('Model Overview')
print(model.summary())
print('Model input shape')
print(model.layers[0].input_shape)

Model Overview
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 36, 9216)          3228864   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                1183872   
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 512)               16896     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 4,430,273
Trainable params: 4,408,321
Non-trainable params: 21,952
_________________________________________________________________
None
Model input shape
(None, 36, 96, 96, 3)


In [4]:
import cv2
from vvadlrs3.utils.imageUtils import cropImage

cap = cv2.VideoCapture(0)
while(True):
    # TODO: this is problematic because it runs on the default framerate but we need 25fps for ideal results
    ret, frame = cap.read()     # Capture frame-by-frame
    dets = detector(frame, 1)   # Detect faces
    if dets:
        features = ffg.getFeatures(cropImage(frame, dets[0]))
        # fill ringbuffer
        rb.append(features)

        if rb.is_full:
            y = model.predict(np.array([rb]))
            s = sample.FeatureizedSample()
            s.data = np.copy(rb)
            s.label = y > 0.5
            s.featureType = featureType
            samples.append(s)
            print(f"added sample {len(samples)}")
        cv2.imshow('frame', frame)
    else:
        # empty ringbuffer - to prevent glitches
        rb.clear()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: array, upsample_num_times: int=0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x7f0adc0c7330>, None, 1

## Record Video

In [5]:
from ipywebrtc import CameraStream, ImageRecorder, VideoRecorder, AudioRecorder
from ipywidgets import Image, HBox
import PIL.Image
import io
import numpy as np
from skimage.filters import sobel
from skimage.color.adapt_rgb import adapt_rgb, each_channel
from skimage import filters
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 },
                        "frameRate": {
                            "min": "1",
                            "max": "1"
                        }})
image_recorder = ImageRecorder(stream=camera)
type(image_recorder.image.value)



bytes

In [20]:
out = Image()
stop_process = False
output = widgets.Output()

@output.capture()
def process_image(_):
    print("new Image")
    if stop_process:
        return
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3]
    dets = detector(im_array, 1)   # Detect faces
    print(dets)
    if dets:
        features = ffg.getFeatures(cropImage(frame, dets[0]))
        # fill ringbuffer
        rb.append(features)

        if rb.is_full:
            y = model.predict(np.array([rb]))
            s = sample.FeatureizedSample()
            s.data = np.copy(rb)
            s.label = y > 0.5
            s.featureType = featureType
            samples.append(s)
            print(f"added sample {len(samples)}")
    image_recorder.recording = True

image_recorder.image.observe(process_image, names=['value'])
#display(image_recorder)
#display(output)

HBox([output, image_recorder])

In [8]:
samples
rb

<RingBuffer of array([], shape=(0, 96, 96, 3), dtype=uint8)>

## Stop processing

In [28]:
stop_process = False

## Clean

In [29]:
from ipywidgets import Widget
Widget.close_all()

In [13]:
from IPython.display import display

class ExampleButton(widgets.Button):
    output = widgets.Output()
    
    @output.capture()
    def on_button_clicked(b):
        print("Button clicked.")
        
button = ExampleButton(description="Click Me!")
display(button)

button.on_click(ExampleButton.on_button_clicked)
display(ExampleButton.output)

ExampleButton(description='Click Me!', style=ButtonStyle())

Output()